In [1]:
import sys
sys.path.insert(0, '..')
import numpy
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
%matplotlib inline
# will autoupdate any of the packages imported:
import pyclifford as pc

In [13]:
from pyclifford.utils import pauli_decompose, calculate_chi, pauli_combine
from pyclifford.paulialg import PauliList, Pauli, PauliPolynomial

In [14]:
class PauliChannel(object):
    '''Pauli channel.
        C[rho] = sum_{m,n} phi_{mn} P_m rho P_n^H,
    where P_m is the m-th element of paulis.
    - The CPTP condition requires phi^H = phi, Tr phi = 1, and phi >=0,
      as if phi is a density matrix.
      
    Parameters:
    paulis: PauliList - a list of Pauli operators {P_m} serving as operator basis.
    phi: complex (L, L) - channel density matrix phi_{mn} in the Pauli basis.'''
    def __init__(self, paulis, phi):
        self.paulis = paulis
        self.phi = phi
        self.N = self.paulis.N

In [15]:
class GeneralizedStabilizerState(object):
    '''Generalized stabilizer state.
        rho = sum_{b,b'} chi_{b,b'} |b> <b'|,
    where |b> is a basis state of destabilizer excitations,
        |b> = prod_{i} D_i^{b_i} |0>,
    with |0> being the state stabilzed by all stabilizers, i.e.
        S_i |0> = |0>.
    - The stabilizers and destabilizers are given by a StabilizerState 
      instance as the stabilizer frame.
    - The basis of destabilizer excitations are represented as
      binary array (tuple), e.g. (0,1,0,1) means the 2nd and 4th 
      destabilizers are excited.
    - The basis dictionary keeps track of the mapping from unique
      binary tuple to integer index of the basis.
    - The density matrix is given by a complex array chi of size L x L,
      where L is the number of excitation basis states.

    Parameters:
    frame: StabilizerState - a stabilizer state serving as the frame.
    basis: int (L, N-r) - a binary array encoding the basis of destabilizer excitations.
    chi: complex (L, L) - density matrix in the excitation basis.'''
    def __init__(self, frame, basis, chi):
        self.frame = frame
        self.basis = basis
        self.chi = chi

    @property
    def N(self):
        return self.frame.N
    
    @property
    def r(self):
        return self.frame.r

    def __repr__(self):
        return f"GeneralizedStabilizerState(\nframe=\n{self.frame},\nbasis=\n{self.basis},\nchi=\n{self.chi})"
    
    def copy(self):
        return GeneralizedStabilizerState(self.frame.copy(), self.basis.copy(), self.chi.copy())
    
    def rotate_by(self, generator, mask=None):
        self.frame.rotate_by(generator, mask)
        return self
    
    def transform_by(self, clifford_map, mask=None):
        self.frame.transform_by(clifford_map, mask)
        return self
    
    def evolve_by(self, pauli_channel):
        # Evolve the state by a Pauli channel.
        bs, cs, ps = pauli_decompose(pauli_channel.paulis.gs, pauli_channel.paulis.ps, 
                                     self.frame.gs, self.frame.ps, self.r)
        # construct new basis, compute fusion map and fusion phase indicator
        L_old = self.basis.shape[0]
        L_add = bs.shape[0]
        L_new = 0
        basis_new = {}
        fusion_map = numpy.zeros((L_old,L_add), dtype=numpy.int_)
        fusion_p = numpy.zeros((L_old,L_add), dtype=numpy.int_)
        for i in range(L_old):
            for j in range(L_add):
                b_new = tuple((self.basis[i] + bs[j])%2)
                if b_new not in basis_new:
                    basis_new[b_new] = L_new
                    L_new += 1
                k = basis_new[b_new]
                fusion_map[i,j] = k
                fusion_p[i,j] = ps[j] + 2*self.basis[i].dot(cs[j])
        # perform fusion of state and channel density matrices
        chi_new = calculate_chi(self.chi, pauli_channel.phi, fusion_map, fusion_p, L_new)
        # in-place update basis and chi
        self.basis = numpy.array(list(basis_new.keys()))
        self.chi = chi_new
        return self
    
    def represent(self, ops):
        '''Represent a list of operators in the destabilizer excitation basis.
        
        Parameters:
        ops: operator(s), can be Pauli, PauliList, PauliPolynomial'''
        if isinstance(ops, Pauli):
            # cast Pauli to PauliPolynomial
            return self.represent(ops.as_polynomial())
        if isinstance(ops, PauliPolynomial):
            # cast PauliPolynomial to PauliList
            mats = self.represent(PauliList(ops.gs,ops.ps))
            return numpy.tensordot(ops.cs, mats, axes=(0,0))
        if isinstance(ops, PauliList):
            bs, cs, ps = pauli_decompose(ops.gs, ops.ps, self.frame.gs, self.frame.ps, self.r)
            L = self.basis.shape[0]
            K = ps.shape[0]
            mats = numpy.zeros((K,L,L), dtype=numpy.complex128)
            idx = {tuple(b): i for i, b in enumerate(self.basis)}
            for k in range(K):
                for i0 in range(L):
                    b0 = self.basis[i0]
                    b1 = tuple((b0 + bs[k])%2)
                    if b1 in idx:
                        i1 = idx[b1]
                        mats[k,i1,i0] = 1j**(ps[k] + 2*b0.dot(cs[k]))
            return mats
            
    def expect(self, obs):
        '''Evaluate expctationvalues of Pauli observables on the generalized
           stabilizer state.
           
        Parameters:
        obs: observable, can be Pauli, PauliList, PauliPolynomial

        Returns:
        out: output (depending on the type of obs)
            * Pauli: promote to PauliPolynomial
            * PauliPolynomial O: Tr(rho O)
            * PauliList [O_i]: [Tr(rho O_i)]''' 
        mats = self.represent(obs)
        if mats.ndim == 2:  # single matrix case
            return numpy.trace(self.chi @ mats)
        else:  # batch case
            return numpy.tensordot(self.chi, mats, axes=([1,0], [1,2]))
        
    def to_numpy(self):
        '''Convert generalized stabilizer state to numpy density matrix representation.'''
        destabilizers = self.frame[self.N+self.r:2*self.N]
        gs, ps =pauli_combine(self.basis, destabilizers.gs, destabilizers.ps)
        Ds = PauliList(gs,ps).to_numpy()
        rho0 = self.frame.to_numpy()
        L = Ds.shape[0]
        rho = numpy.zeros_like(rho0)
        for j1 in range(L):
            for j2 in range(L):
                rho += self.chi[j1, j2]* Ds[j1] @ rho0 @ Ds[j2]
        return rho
                

Test: create a generalized stabilizer state over three qubits, where the first two qubits is in a mixture of Bell state, and the third qubit is maximally mixed.

In [25]:
state=GeneralizedStabilizerState(
    pc.stabilizer_state("XXI","ZZI"), 
    numpy.array([[0,0],[1,0]]), 
    numpy.array([[0.8,0.1],[0.1,0.2]],dtype=numpy.complex128))
state

GeneralizedStabilizerState(
frame=
StabilizerState(
   +XXI
   +ZZI),
basis=
[[0 0]
 [1 0]],
chi=
[[0.8+0.j 0.1+0.j]
 [0.1+0.j 0.2+0.j]])

Density matrix of the state.

In [24]:
state.to_numpy().real

array([[0.4 , 0.  , 0.  , 0.  , 0.05, 0.  , 0.  , 0.  ],
       [0.  , 0.4 , 0.  , 0.  , 0.  , 0.05, 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.05, 0.  , 0.  , 0.  , 0.1 , 0.  , 0.  , 0.  ],
       [0.  , 0.05, 0.  , 0.  , 0.  , 0.1 , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ]])

Represent a list of Pauli operators on the stabilizer frame basis.

In [18]:
state.represent(pc.paulis("XXI","XYI","YXI","YYI"))

array([[[ 1.+0.j,  0.+0.j],
        [ 0.+0.j, -1.+0.j]],

       [[ 0.+0.j,  0.+1.j],
        [-0.-1.j,  0.+0.j]],

       [[ 0.+0.j,  0.+1.j],
        [-0.-1.j,  0.+0.j]],

       [[-1.+0.j,  0.+0.j],
        [ 0.+0.j,  1.+0.j]]])

Compute the expectation value of a Pauli polynomial on the state.

In [26]:
obs = (pc.pauli("XXI")+1j*pc.pauli("XYI")+1j*pc.pauli("YXI")-pc.pauli("YYI"))/4
state.expect(obs)

np.complex128(0.3+0j)

Verify that the result is correct by explicit calculation using matrix representations of state and operator.

In [27]:
numpy.trace(state.to_numpy() @ obs.to_numpy())

np.complex128(0.30000000000000004+0j)

Define a Pauli channel which maximally decohere the first qubit in the Z basis, then apply the channel to the state.

In [20]:
channel = PauliChannel(pc.paulis("III","ZII"), numpy.array([[0.5,0.0],[0.0,0.5]],dtype=numpy.complex128))
state.evolve_by(channel)

GeneralizedStabilizerState(
frame=
StabilizerState(
   +XXI
   +ZZI),
basis=
[[0 0]
 [1 0]],
chi=
[[0.5+0.j 0.1+0.j]
 [0.1+0.j 0.5+0.j]])

The density matrix shows that the decoherence indeed removes off-diagonal matrix elements.

In [21]:
state.to_numpy().real

array([[0.3, 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0.3, 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0.2, 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.2]])